# Urban Tree Canopy - Part 1 (LiDAR Extractor)
## When new LiDAR data is available, this script scrapes the neccesary LiDAR files from TxGIO, processes them into a polygon shapefile, and is saved into a newly created GDB. After running this script, you will need to manually QA/QC the final shapefile to check for accuracy and edit as needed. After QA/QC is complete, you can run Part 2.
## BEFORE YOU BEGIN
 1. Download the ETJ, City Layer, CHHD Layers from sharepoint
 2. Import the data into ArcGIS Pro (ensure its in your contents pane)
 3. Download the LiDAR Index from TxGIO for the dataset you are going to use
 4. Find the find the cities that are completely within the available LiDAR Data. 
 5. Export these cities to a new shapefile and save it to a GDB
 6. The ONLY THING you need to do are these steps and define the variables below, ensuring your file and folder names are where you want them to go. Change the base_url to the S3 link provided on the TxGIO webiste, all the way until the lpc/. (e.g. r'https://tnris-data-warehouse.s3.us-east-1.amazonaws.com/LCD/collection/stratmap-2023-35cm-50cm-elpaso-clearfork-brazos/lpc/'

In [ ]:
#DONT CHANGE THESE
import os
import requests
import tempfile
arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "80%"
city_names_field = 'Name10'

#CHANGE THESE
base_url = r'https://tnris-data-warehouse.s3.us-east-1.amazonaws.com/LCD/collection/stratmap-2021-28cm-50cm-bexar-travis/lpc/'
workspace = r'D:\ArcGIS_Projects\UTC\UTC'
#CHANGE THESE ALSO. USE THE DISPLAY NAME POINTING TO THE SHAPEFILE. 
#DO NOT USE THE FULL PATH POINTING TO GDB, OTHERWISE THE CODE WILL NOT WORK
ETJ = 'Buda_ETJ'
lidar_layer = 'Lidar_2021_2023_Files'


#DONT CHANGE THESE
qa_qc_gdb_name = f"Polygons_from_{ETJ}"
qa_qc_gdb_path = os.path.join(workspace, f'{qa_qc_gdb_name}.gdb')

In [ ]:
def get_city_names(etj_layer, city_field):
    city_names = []
    with arcpy.da.SearchCursor(etj_layer, city_field) as cursor:
        for row in cursor:
            city_names.append(row[0])
    return city_names

def update_city_names(etj_layer, city_field):
    with arcpy.da.UpdateCursor(etj_layer, [city_field]) as cursor:
        for row in cursor:
            row[0] = row[0].replace(" ", "")
            cursor.updateRow(row)
    print(f"Updated city names in {etj_layer}")
    
def get_ninth_column_name(layer):
    fields = arcpy.ListFields(layer)
    if len(fields) >= 9:
        return fields[8].name
    else:
        raise ValueError("The layer does not have at least 9 columns.")

def download_and_convert_lidar_files(city, lidar_layer, base_url, folder_path):
    file_list = []
    with arcpy.da.SearchCursor(lidar_layer, 'tilename') as cursor:
        for row in cursor:
            file_list.append(f"{row[0].lower()}.laz")
    print(f"{city} has {len(file_list)} laz files")
    
    for count, file_name in enumerate(file_list, start=1):
        local_filename = os.path.join(folder_path, file_name)
        full_url = base_url + file_name
        print(full_url)
    
        if os.path.exists(local_filename):
            print(f"File {file_name} already exists, skipping download.")
        else:
            try:
                with requests.get(full_url, stream=True) as r:
                    r.raise_for_status()
                    with open(local_filename, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192):
                            f.write(chunk)
                arcpy.conversion.ConvertLas(local_filename, folder_path)
                print(f"{count} / {len(file_list)} downloaded and converted to LAS")
            except Exception as e:
                print(f"Error downloading {file_name}: {e}")

def process_lidar_files(city, folder_path, workspace):
    las_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.las')]
    lasd = arcpy.management.CreateLasDataset(las_files, os.path.join(folder_path, f'{city}.lasd'))
    
    filtered_lasd = arcpy.management.MakeLasDatasetLayer(lasd, f'{city}_FilteredLASD', 5, ["FIRST_OF_MANY", "SINGLE", 1])
    
    lasd_raster = arcpy.conversion.LasDatasetToRaster(filtered_lasd, f'{city}_LASD_to_Raster.tif', 'ELEVATION', 'BINNING MAXIMUM SIMPLE', 'INTEGER', 'CELLSIZE', 1, 1)
    
    min_value = arcpy.GetRasterProperties_management(lasd_raster, "MINIMUM").getOutput(0)
    max_value = arcpy.GetRasterProperties_management(lasd_raster, "MAXIMUM").getOutput(0)
    print(f"Min value: {min_value}, Max value: {max_value}")
    
    remap_range = arcpy.sa.RemapRange([[min_value, max_value, 1]])
    reclassed_raster = arcpy.sa.Reclassify(lasd_raster, "VALUE", remap_range)
    reclassed_raster.save(f'{city}_reclassed.tif')
    
    raster_to_polygon_path = f"{city}_Raster2Polygon"
    arcpy.conversion.RasterToPolygon(reclassed_raster, raster_to_polygon_path)
    
    
    arcpy.management.AddFields(raster_to_polygon_path, [['area', 'FLOAT'], ['perimeter', 'FLOAT'], ['PeriArea', 'FLOAT']])
    arcpy.management.CalculateGeometryAttributes(raster_to_polygon_path, [['area', 'AREA_GEODESIC']], 'METERS', 'SQUARE_METERS')
    arcpy.management.CalculateGeometryAttributes(raster_to_polygon_path, [['perimeter', 'PERIMETER_LENGTH_GEODESIC']], 'METERS', 'SQUARE_METERS')
    arcpy.management.CalculateField(raster_to_polygon_path, 'PeriArea', '!perimeter! / !area!')
    
    arcpy.SelectLayerByAttribute_management(raster_to_polygon_path,'NEW_SELECTION', 'PeriArea > 1.3')
    arcpy.DeleteFeatures_management(raster_to_polygon_path)
    arcpy.SelectLayerByAttribute_management(raster_to_polygon_path,'CLEAR_SELECTION')
    
    union = f'{city}_Union'
    arcpy.analysis.Union([raster_to_polygon_path, raster_to_polygon_path], union, 'ALL', "", "NO_GAPS")
    arcpy.management.AddField(union, 'area2', 'FLOAT')
    arcpy.management.CalculateGeometryAttributes(union, [['area2', 'AREA_GEODESIC']], 'METERS', 'SQUARE_METERS')
    
    ninth_column_name = get_ninth_column_name(union)
    arcpy.management.SelectLayerByAttribute(union, "NEW_SELECTION", f"area2 > 7 AND {ninth_column_name} = -1", None)
    arcpy.DeleteFeatures_management(union)
    
    arcpy.management.SelectLayerByAttribute(union, "NEW_SELECTION", f"area2 < 5", None)
    arcpy.DeleteFeatures_management(union)
    
    merge = f'{city}'
    arcpy.management.Merge(union, merge)
    
    
    
    output_fc = os.path.join(qa_qc_gdb_path, merge)
    arcpy.management.CopyFeatures(merge, output_fc)
    
    print(f"Processed lidar files for {city}")


def cleanup_las_files(directory):
    for file_name in os.listdir(directory):
        if file_name.endswith('.las') or file_name.endswith('.laz'):
            file_path = os.path.join(directory, file_name)
            try:
                os.remove(file_path)
                print(f"Deleted: {file_path}")
            except Exception as e:
                print(f"Error deleting {file_path}: {e}")

    
def main():
    update_city_names(ETJ, city_names_field)
    city_names = get_city_names(ETJ, city_names_field)
    arcpy.CreateFileGDB_management(workspace, qa_qc_gdb_name)
    print(city_names)
    for city in city_names:
        temp_dir = arcpy.env.scratchFolder
        folder_path = arcpy.CreateFolder_management(temp_dir, city).getOutput(0)
        arcpy.env.workspace = folder_path
        
        arcpy.management.SelectLayerByAttribute(ETJ, "NEW_SELECTION", f'"Name10" = \'{city}\'')
        arcpy.management.SelectLayerByLocation(lidar_layer, 'INTERSECT', ETJ)
        
        download_and_convert_lidar_files(city, lidar_layer, base_url, folder_path)
        process_lidar_files(city, folder_path, workspace)
        
        arcpy.management.SelectLayerByAttribute(ETJ, "CLEAR_SELECTION")
        arcpy.management.SelectLayerByAttribute(lidar_layer, "CLEAR_SELECTION")
        
        cleanup_las_files(folder_path)
        
        print(f"Completed processing for {city}\n")
        


In [ ]:
if __name__ == "__main__":
    main()